In [ ]:
import requests
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import numpy as np
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from functools import partial
import csv

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('https://', adapter)
def Crawl_THPTQG(year, so_bao_danh):
    so_bao_danh = str(so_bao_danh).rjust(8, '0')
    year = str(year)
    URL = "https://vietnamnet.vn/giao-duc/diem-thi/tra-cuu-diem-thi-tot-nghiep-thpt/{}/{}.html".format(year, so_bao_danh)
    r = session.get(URL)
    subjects = ['Toán', 'Văn', 'Ngoại ngữ', 'Lí', 'Hóa', 'Sinh', 'Sử', 'Địa', 'GDCD']
    scores = [np.nan]*(len(subjects)+1)
    scores[0] = so_bao_danh
    subjects_idx = {sub: idx+1 for idx, sub in enumerate(subjects)}
    if r.status_code != 404:
        soup = BeautifulSoup(r.content, 'html.parser')
        target = soup.find('div', attrs={'class': 'resultSearch__right'})
        table = target.find('tbody')
        rows = table.find_all('tr')
        placeHolder = []
        for row in rows:
            lst = row.find_all('td')
            cols = [ele.text.strip() for ele in lst]
            # placeHolder.append([ele for ele in cols if ele])
            scores[subjects_idx[cols[0]]] = cols[1]
        # content = "{},{},{}\n".format(year, so_bao_danh, placeHolder)
    else:
        return None
    return scores



if __name__ == "__main__":
    provinces = range(50,51)
    years = [2021]
    for year in years:
        for province in provinces:
            count = 1
            start = province*1000000
            # lst = [i for i in range(start, start + 1000000)]
            lst = range(start, start + 150000)
            NUM_WORKERS = cpu_count() * 8
            pool = Pool(NUM_WORKERS)
            year_func_partial = partial(Crawl_THPTQG, year)
            result_iter = pool.imap(year_func_partial, lst)
            with open(f"thptqg_{year}.csv", "a", encoding='utf-8') as f:
                writer_csv = csv.writer(f)
                for result in result_iter:
                    if result is not None:
                        writer_csv.writerow(result)
                        print("Đang xử lý {} với số báo danh: {}".format(year, result[0]))
                    else:
                        count += 1
                        if count == 10:
                            break
            # with open(f"thptqg_{year}.csv", "a", encoding='utf-8') as f:
            #         if result_iter is not None:
            #             writer_csv = csv.writer(f)
            #             writer_csv.writerow(result_iter)
            pool.close()
            pool.join()
